In [1]:
from __future__ import print_function
import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv3D, MaxPooling3D
from keras.optimizers import SGD
from keras.layers.normalization import BatchNormalization

import os
from keras import backend as K

from sklearn.model_selection import train_test_split
import numpy as np

landmarks = [
    'PCLOrigin',
    'lateralCondyle',
    'medialCondyle',
    'medialSulcus',
    'whitesideReference'
]


def one_axis_femur_model(name='PCLOrigin', axis=0):
    output_path = "/code/" + name + "/"

    data_size = 15
    imgs_to_process = []
    lbls_to_process = []
    dif_arr = [(0,1,2), (0,2,1), (1,0,2), (1,2,0), (2,0,1), (2,1,0)]
    num_classes = 120

    for index in range(data_size):
        for a, b, c in dif_arr:
            imgs_to_process.append(np.load(output_path + 'images/' + '{}-{}-{}-{}.npy'.format(index, a, b, c)))
            label = np.load(output_path + 'label/' + '{}-{}-{}-{}.npy'.format(index, a, b, c))
            lbls_to_process.append(int(label[3+axis]))
            num_classes = int(label[6]) * 2

    imgs_to_process = np.array(imgs_to_process)
    lbls_to_process = np.array(lbls_to_process)

    batch_size = 15
    epochs = 200
    learning_rate = 0.001

    x_train, x_test, y_train, y_test = train_test_split(imgs_to_process, lbls_to_process, test_size=0.2)
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')
    img_rows = x_train.shape[1]
    img_cols = x_train.shape[2]
    colors = x_train.shape[3]


    if K.image_data_format() == 'channels_first':
        x_train = x_train.reshape(x_train.shape[0], 1,colors, img_rows, img_cols)
        x_test = x_test.reshape(x_test.shape[0], 1,colors, img_rows, img_cols)
        input_shape = (1, colors, img_rows, img_cols)
    else:
        x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, colors, 1)
        x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, colors, 1)
        input_shape = (img_rows, img_cols, colors, 1)

    print(num_classes)
    # Convert class vectors to binary class matrices.
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)

    model = load_model('one_axis_femur_model.h5')

    score = model.evaluate(x_test, y_test, verbose=0)
    prediction = model.predict(x_test)
    print(prediction)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])
    print(score)
    print(y_test)

Using TensorFlow backend.


In [2]:
one_axis_femur_model('medialSulcus', 0)

x_train shape: (72, 42, 42, 42)
72 train samples
18 test samples
72 train samples
18 test samples
42
[[3.51604191e-22 3.04960498e-32 6.90163531e-31 2.30202819e-36
  5.96277332e-19 1.04776609e-32 5.08250561e-31 2.17175769e-20
  8.26891251e-30 1.01382146e-27 5.15053787e-26 2.40995096e-26
  8.04922113e-22 1.03253505e-22 7.93059485e-38 1.81185964e-28
  1.32004151e-32 3.09276246e-26 3.10586687e-23 2.72518409e-24
  4.34600433e-31 1.11338288e-22 2.24094229e-22 1.08802410e-20
  8.88393692e-10 7.19802560e-18 1.56712598e-19 1.00000000e+00
  1.05970363e-11 1.32262623e-26 2.29079543e-23 2.82758912e-31
  2.21491450e-26 1.49260988e-26 0.00000000e+00 1.54513957e-33
  6.49916420e-30 1.48027351e-23 3.97585403e-26 3.65963184e-24
  9.22382487e-31 2.14626982e-18]
 [1.54893474e-17 1.74771308e-21 2.28031344e-17 6.70518201e-24
  1.03633937e-21 1.64409253e-22 1.57400003e-16 6.93601335e-19
  2.13764574e-21 9.63620251e-27 1.44969871e-17 8.67260098e-20
  1.16001206e-21 1.70718752e-22 1.75949511e-19 6.26259051e-1